In [1]:
!pip install EMD-signal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.42.0
    Uninstalling tqdm-4.42.0:
      Successfully uninstalled tqdm-4.42.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False

In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import joblib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from PyEMD import EEMD
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

# ===============================
# 1️⃣ Load Data & Basic Preprocessing
# ===============================
print("1️⃣ Loading Data...")
df = pd.read_csv("full_data_imputed.csv", index_col=False)

# Clean up columns
drop_cols = ['Unnamed: 0', 'Latitude', 'Longitude']
df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)

df_copy = df.copy()
df_copy['Local Time'] = pd.to_datetime(df_copy['Local Time'])
df_copy = df_copy.set_index('Local Time')

# Fill missing values
weather_cols = ['Clouds', 'Precipitation', 'Pressure', 'Relative Humidity', 'Temperature', 'UV_Index', 'Wind Speed']
existing_cols = [col for col in weather_cols if col in df_copy.columns]
df_copy[existing_cols] = df_copy[existing_cols].ffill().bfill()

target_col = 'Aqi'

# ===============================
# 2️⃣ EEMD Preprocessing (TURBO MODE: Parallel + Chunking)
# ===============================
print("\n2️⃣ Running Parallel EEMD (This may take 20-40 mins)...")

# --- Cấu hình ---
trials = 50       # Số lần thử (Giảm từ 100 xuống 50 để tăng tốc độ gấp đôi)
noise_width = 0.2 
K_target = 9      # Giữ lại đúng 9 modes
CHUNK_SIZE = 2000 # Cắt nhỏ dữ liệu để xử lý song song

signal = df_copy[target_col].values
n_samples = len(signal)

# --- Hàm xử lý 1 chunk (Chạy trên 1 core CPU) ---
def process_single_chunk(chunk_idx, start_idx, end_idx, signal_chunk, trials, noise, k_target):
    try:
        # Init EEMD
        eemd = EEMD(trials=trials, noise_width=noise)
        # Run EEMD
        imfs = eemd.eemd(signal_chunk)
        
        # --- Fix Shape & Modes ---
        current_k, length = imfs.shape
        result_matrix = np.zeros((k_target, length))
        
        if current_k >= k_target:
            result_matrix = imfs[:k_target, :]
        else:
            result_matrix[:current_k, :] = imfs
            
        return (chunk_idx, result_matrix.T)
        
    except Exception as e:
        # Return zeros on error to prevent crash
        return (chunk_idx, np.zeros((end_idx - start_idx, k_target)))

# --- Tạo danh sách tasks ---
tasks = []
for i in range(0, n_samples, CHUNK_SIZE):
    end_idx = min(i + CHUNK_SIZE, n_samples)
    chunk_signal = signal[i:end_idx]
    if len(chunk_signal) > 50: # Bỏ qua đoạn quá ngắn
        tasks.append((i // CHUNK_SIZE, i, end_idx, chunk_signal))

# --- Chạy song song (n_jobs=-1: Dùng tất cả CPU) ---
results = Parallel(n_jobs=-1)(
    delayed(process_single_chunk)(idx, start, end, sig, trials, noise_width, K_target)
    for idx, start, end, sig in tqdm(tasks, desc="EEMD Progress")
)

# --- Ghép dữ liệu ---
print("   -> Merging chunks...")
results.sort(key=lambda x: x[0])
final_imfs = np.concatenate([res[1] for res in results], axis=0)

# Padding nếu thiếu dòng cuối
if len(final_imfs) < len(df_copy):
    missing = len(df_copy) - len(final_imfs)
    padding = np.zeros((missing, K_target))
    final_imfs = np.vstack([final_imfs, padding])

# Thêm vào DataFrame
for i in range(K_target):
    df_copy[f'IMF_{i+1}'] = final_imfs[:, i]

print(f"✅ Added {K_target} IMF columns to dataset.")

# ===============================
# 3️⃣ Feature Engineering
# ===============================
print("\n3️⃣ Feature Engineering...")
df_copy['Month'] = df_copy.index.month
df_copy['Day'] = df_copy.index.day
df_copy['Hour'] = df_copy.index.hour
df_copy['Weekday'] = df_copy.index.weekday

# ===============================
# 4️⃣ Split Data
# ===============================
train_data = df_copy.loc['2023':'2024']
val_data = df_copy.loc['2025-01':'2025-06']
test_data = df_copy.loc['2025-07':'2025-09']

X_train, y_train = train_data.drop(columns=[target_col]), train_data[target_col]
X_val, y_val = val_data.drop(columns=[target_col]), val_data[target_col]
X_test, y_test = test_data.drop(columns=[target_col]), test_data[target_col]

# ===============================
# 5️⃣ Scaling
# ===============================
print("5️⃣ Scaling Data...")
scaler_X = StandardScaler()
# Lưu ý: X đã bao gồm các cột IMF nên chúng cũng được scale
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_val_scaled = scaler_y.transform(y_val.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# ===============================
# 6️⃣ Create Datasets
# ===============================
length = 24  # 24 hours window
batch_size = 32
n_features = X_train_scaled.shape[1]

train_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    X_train_scaled, targets=y_train_scaled, sequence_length=length, batch_size=batch_size, shuffle=False)

val_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    X_val_scaled, targets=y_val_scaled, sequence_length=length, batch_size=batch_size, shuffle=False)

test_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    X_test_scaled, targets=y_test_scaled, sequence_length=length, batch_size=batch_size, shuffle=False)

# ===============================
# 7️⃣ Model Definition
# ===============================
def rmsle_custom(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return tf.sqrt(msle(y_true, y_pred))

def r2_keras(y_true, y_pred):
    y_pred = tf.cast(y_pred, y_true.dtype)
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - ss_res / (ss_tot + K.epsilon())

model = Sequential()
# CNN Layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(length, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
# LSTM Layers
model.add(LSTM(units=100, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False, activation='tanh'))
model.add(Dropout(0.2))
# Output
model.add(Dense(units=1))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=[rmsle_custom, 'mae', 'mse', tf.keras.metrics.RootMeanSquaredError(name='rmse'), 'mape', 'msle', r2_keras]
)

1️⃣ Loading Data...

2️⃣ Running Parallel EEMD (This may take 20-40 mins)...


EEMD Progress:   0%|          | 0/471 [00:00<?, ?it/s]

   -> Merging chunks...
✅ Added 9 IMF columns to dataset.

3️⃣ Feature Engineering...
5️⃣ Scaling Data...


In [3]:
# ===============================
# 8️⃣ Training
# ===============================
print("\n8️⃣ Training Model...")
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1, min_lr=1e-6)

hist = model.fit(train_dataset, validation_data=val_dataset, epochs=50, callbacks=[es, plateau], verbose=1)


8️⃣ Training Model...
Epoch 1/50
21421/21421 [==============================] - 143s 6ms/step - loss: 0.0387 - rmsle_custom: 0.0324 - mae: 0.1179 - mse: 0.0387 - rmse: 0.1966 - mape: 76.1702 - msle: 0.0055 - r2_keras: -1.4181 - val_loss: 0.0275 - val_rmsle_custom: 0.0439 - val_mae: 0.1250 - val_mse: 0.0275 - val_rmse: 0.1657 - val_mape: 98.1339 - val_msle: 0.0050 - val_r2_keras: -0.1070 - lr: 0.0010
Epoch 2/50
21421/21421 [==============================] - 139s 7ms/step - loss: 0.0180 - rmsle_custom: 0.0225 - mae: 0.0844 - mse: 0.0180 - rmse: 0.1341 - mape: 47.0983 - msle: 0.0018 - r2_keras: -0.5977 - val_loss: 0.0194 - val_rmsle_custom: 0.0344 - val_mae: 0.1053 - val_mse: 0.0194 - val_rmse: 0.1391 - val_mape: 74.1165 - val_msle: 0.0030 - val_r2_keras: 0.0417 - lr: 0.0010
Epoch 3/50
21421/21421 [==============================] - 139s 7ms/step - loss: 0.0162 - rmsle_custom: 0.0209 - mae: 0.0799 - mse: 0.0162 - rmse: 0.1273 - mape: 43.3626 - msle: 0.0015 - r2_keras: 0.1271 - val_loss: 0

In [4]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error

# ===============================
# 🔟 Dự đoán & Inverse Transform
# ===============================
print("\n------------------------------------------------")
print("Đang tính toán Full Metrics trên tập Test...")
print("------------------------------------------------")

# 1. Dự đoán (Output đang ở dạng Scaled)
y_pred_scaled = model.predict(test_dataset, verbose=0)

# 2. Lấy y_true (Output đang ở dạng Scaled) từ dataset
y_true_scaled = np.concatenate([y for x, y in test_dataset], axis=0)

# 3. Đưa về giá trị gốc (Original Scale)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_true_scaled)

# 4. Xử lý giá trị âm (Quan trọng cho MSLE/RMSLE)
# AQI không thể âm, nhưng model có thể dự đoán ra số âm nhỏ.
# Ta cần đưa về 0 để tránh lỗi logarit.
y_pred_safe = np.maximum(y_pred, 0)
y_true_safe = np.maximum(y_true, 0)

# ===============================
# 1️⃣1️⃣ Tính toán 7 Metrics
# ===============================

# 1. MAE
mae = mean_absolute_error(y_true, y_pred)

# 2. MSE
mse = mean_squared_error(y_true, y_pred)

# 3. RMSE
rmse = np.sqrt(mse)

# 4. MSLE (Mean Squared Logarithmic Error)
msle = mean_squared_log_error(y_true_safe, y_pred_safe)

# 5. RMSLE (Root Mean Squared Logarithmic Error)
rmsle = np.sqrt(msle)

# 6. MAPE (Mean Absolute Percentage Error)
# Thêm epsilon (1e-7) để tránh chia cho 0
mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-7))) * 100

# 7. R-squared
r2 = r2_score(y_true, y_pred)

# ===============================
# 1️⃣2️⃣ In kết quả Report
# ===============================
print("\n" + "="*50)
print(f"📊 REPORT ĐÁNH GIÁ MÔ HÌNH VMD-CNN-LSTM")
print("="*50)
print(f"{'Metric':<10} | {'Giá trị':<15} | {'Ý nghĩa ngắn gọn'}")
print("-" * 50)
print(f"{'MAE':<10} | {mae:<15.4f} | Sai số tuyệt đối trung bình (đơn vị AQI)")
print(f"{'MSE':<10} | {mse:<15.4f} | Bình phương sai số (phạt lỗi lớn nặng hơn)")
print(f"{'RMSE':<10} | {rmse:<15.4f} | Sai số chuẩn (cùng đơn vị với AQI)")
print(f"{'MSLE':<10} | {msle:<15.4f} | Sai số logarit bình phương")
print(f"{'RMSLE':<10} | {rmsle:<15.4f} | Sai số logarit (tốt nếu dữ liệu chênh lệch lớn)")
print(f"{'MAPE':<10} | {mape:<15.2f}% | Sai số theo phần trăm")
print(f"{'R²':<10} | {r2:<15.4f} | Độ phù hợp (Max = 1.0)")
print("="*50)


------------------------------------------------
Đang tính toán Full Metrics trên tập Test...
------------------------------------------------

📊 REPORT ĐÁNH GIÁ MÔ HÌNH VMD-CNN-LSTM
Metric     | Giá trị         | Ý nghĩa ngắn gọn
--------------------------------------------------
MAE        | 13.0486         | Sai số tuyệt đối trung bình (đơn vị AQI)
MSE        | 254.6857        | Bình phương sai số (phạt lỗi lớn nặng hơn)
RMSE       | 15.9589         | Sai số chuẩn (cùng đơn vị với AQI)
MSLE       | 0.1964          | Sai số logarit bình phương
RMSLE      | 0.4432          | Sai số logarit (tốt nếu dữ liệu chênh lệch lớn)
MAPE       | 48.78          % | Sai số theo phần trăm
R²         | 0.7310          | Độ phù hợp (Max = 1.0)


Even the multi-class problem can be solved near perfectly with XGBoost.  

# Conclusion 

- As we saw in the binary and multi-class classification problems, XGBoost can be very effective at detecting anomalies when you have labeled data. In labs 2 and 3, we will consider the same KDD99 dataset but we will train deep learning models to detect anomalies without using the labels.  This will mimic a more likely situation is the real world.

- GPU-Accelerating XGboost through RAPIDS is easy and fast.  The only change we had to make to use the GPU was to set the 'tree_method' parameter to 'gpu_hist'.

# References

<ol>
<li>
Dhaliwal, S., Nahid, A., & Abbas, R. (2018). Effective Intrusion Detection System Using XGBoost. Information, 9(7), 149. doi:10.3390/info9070149
</li>
<li>
Brownlee, J. A Gentle Introduction to XGBoost for Applied Machine Learning. Machine Learning
Mastery. Available online: http://machinelearningmastery.com/gentle-introduction-xgboost-applied-machine-learning/
(accessed on 2 March 2018).
    </li>
    <li>A Study on NSL-KDD Dataset for Intrusion Detection System Based on Classification Algorithms. Available
online: https://pdfs.semanticscholar.org/1b34/80021c4ab0f632efa99e01a9b073903c5554.pdf (accessed on
        26 March 2018)</li>
    <li>
        XGBoost Parameters—Xgboost 0.7 Documentation. Available online: http://xgboost.readthedocs.io/en/
latest/parameter.html (accessed on 12 March 2018)
    </li>
    <li>
        RAPIDS Documentation and Cheat Sheet.Available online: https://rapids.ai/documentation.html
    </li>
    
   

<a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>